Python 3.9.13
OS: Windows 

#                                            HOW TO CLEAN DIRTY DATA

### Content

The dataset use for this exercise was taken from Kaggle and has following fields:

* `country`
* `country_code`
* `date_added`
* `has_expired` - Always `false`.
* `job_description` - The primary field for this dataset, containing the bulk of the information on what the job is about.
* `job_title`
* `job_type` - The type of tasks and skills involved in the job. For example, "management".
* `location`
* `organization`
* `page_url`
* `salary`
* `sector` - The industry sector the job is in. For example, "Medical services".

### I generally classify dirty data into 2 categories: Structure Dirty (SD) and Content Dirty (CD)

In [66]:
# First step load libraries
import pandas as pd
import pyarrow as pa
import numpy as np
import pyarrow.parquet as pq
from pyarrow import csv, parquet

### Lets load CVS file to work with it

In [67]:

df = pd.read_csv("Data/monster_com-job_sample.csv",encoding="utf-8")

In [68]:
type(df)
# Lets find out the type of the dataset,offcourse it has to  be pandas

pandas.core.frame.DataFrame

In [69]:
shape_=df.shape
print(shape_)
# We can see that we have 22000 rows and 14 columns, this information is very useful when you clean data because allow you to check changes

(22000, 14)


In [70]:
df.isnull().sum()/shape_[0]*100
# With this method we identify the quantity of null values
# Acording to our first shape command we get 22000 rows 
# lets findout the procentage of empty values

country             0.000000
country_code        0.000000
date_added         99.445455
has_expired         0.000000
job_board           0.000000
job_description     0.000000
job_title           0.000000
job_type            7.400000
location            0.000000
organization       31.213636
page_url            0.000000
salary             84.336364
sector             23.609091
uniq_id             0.000000
dtype: float64

In [71]:
df = df.set_index('uniq_id')
df.head(5)
# Before doing anything we are going to ser the column 'uniq_id' as an index, this way we are noi going to have integrity issues when makin join o merge

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,page_url,salary,sector
uniq_id,,,,,,,,,,,,,
11d599f229a80023d2f40e7c52cd941e,United States of America,US,NaN,No,jobs.monster.com,TeamSoft is seeing an IT Support Specialist to...,IT Support Technician Job in Madison,Full Time Employee,"Madison, WI 53702",NaN,http://jobview.monster.com/it-support-technici...,NaN,IT/Software Development
e4cbb126dabf22159aff90223243ff2a,United States of America,US,NaN,No,jobs.monster.com,The Wisconsin State Journal is seeking a flexi...,Business Reporter/Editor Job in Madison,Full Time,"Madison, WI 53708",Printing and Publishing,http://jobview.monster.com/business-reporter-e...,NaN,NaN
839106b353877fa3d896ffb9c1fe01c0,United States of America,US,NaN,No,jobs.monster.com,Report this job About the Job DePuy Synthes Co...,Johnson & Johnson Family of Companies Job Appl...,"Full Time, Employee",DePuy Synthes Companies is a member of Johnson...,Personal and Household Services,http://jobview.monster.com/senior-training-lea...,NaN,NaN
58435fcab804439efdcaa7ecca0fd783,United States of America,US,NaN,No,jobs.monster.com,Why Join Altec? If you’re considering a career...,Engineer - Quality Job in Dixon,Full Time,"Dixon, CA",Altec Industries,http://jobview.monster.com/engineer-quality-jo...,NaN,Experienced (Non-Manager)
64d0272dc8496abfd9523a8df63c184c,United States of America,US,NaN,No,jobs.monster.com,Position ID# 76162 # Positions 1 State CT C...,Shift Supervisor - Part-Time Job in Camphill,Full Time Employee,"Camphill, PA",Retail,http://jobview.monster.com/shift-supervisor-pa...,NaN,Project/Program Management


##### Lets to drop columns that has no information an are no really necesary, in this case salary is not going to be erase despite of the high porcentage of missing data because can be use later as a lable column

- 'date_added' column has 99.45 of null data so is necesary to drop it

In [72]:
# each of These columns have the same information so we can take for granted that are nos usefull, erasing these columns we can get better performance.

df['country_code'].unique() # the unique value for country code is US 

df['job_board'].unique() # The unique value for  Job_board jobs.monster.com

df['has_expired'].unique() # The unique value form has_expired  is No

df['page_url'].unique() #Page_url has the same information of job_title

df['country'].unique() # The unique value for country is United States of America

array(['United States of America'], dtype=object)

In [73]:
df = df.drop(['date_added','page_url','country_code','has_expired','country','job_board'], axis = 1)
df.tail(5)

# Now lets drop each one of the columns that have the same information

,job_description,job_title,job_type,location,organization,salary,sector
uniq_id,,,,,,,
a80bc8cc3a90c17eef418963803bc640,This is a major premier Cincinnati based finan...,Assistant Vice President - Controller Job in C...,Full Time,"Cincinnati, OH",NaN,"120,000.00 - 160,000.00 $ /yearbonus",NaN
419a3714be2b30a10f628de207d041de,Luxury homebuilder in Cincinnati seeking multi...,Accountant Job in Cincinnati,Full Time,"Cincinnati, OH 45236",Construction - Residential & Commercial/Office,"45,000.00 - 60,000.00 $ /year",Manager (Manager/Supervisor of Staff)
5a590350b73b2cec46b05750a208e345,RE: Adobe AEM- Client - Loca...,AEM/CQ developer Job in Chicago,Full Time,"Chicago, IL 60602",NaN,NaN,NaN
40161cf61c283af9dc2b0a62947a5f1b,Jernberg Industries was established in 1937 an...,Electrician - Experienced Forging Electrician ...,Full Time Employee,"Chicago, IL 60609","Jernberg Industries, Inc.",25.00 - 28.00 $ /hour,Installation/Maintenance/Repair
cb49f16ad72627b109e434e0cac97f7a,Contract AdministratorCan you be the point per...,Contract Administrator Job in Cincinnati,Full Time,"Cincinnati, OH",NaN,"40,000.00 - 46,000.00 $ /year+ annual bonus (u...",Experienced (Non-Manager)


In [74]:
 df = df.dropna(subset=['job_type','organization','sector'], how='any') 
 df.tail(10)
 # Now lets drop a subset of Na data of this three columns that is not significative

,job_description,job_title,job_type,location,organization,salary,sector
uniq_id,,,,,,,
8e9f8638556bc1fd671bb99f4f01ac4d,CULINARY CAREER WEST CHESTER OHIO!EXECUTIVE CH...,EXECUTIVE CHEF WEST CHESTER OHIO $K-$K PLUS! B...,Full Time,"West Chester, OH",All,"75,000.00 - 85,000.00 $ /yearHighly Competitiv...",Manager (Manager/Supervisor of Staff)
ebce61a714f4dd7d15b0263fab42751e,"McCormick & Company, Incorporated, a global le...",Customer Business Manager Job in Cincinnati,"Full Time, Employee","Cincinnati, OH 45202",Food and Beverage Production,NaN,Marketing/Product
8a36252a31d7b06e901be0596bb6501a,About the JobWhat is more secure than the repl...,Outside Sales Representative Job in Cincinnati,Full Time Employee,"Cincinnati, OH 45202",AllEnergy and UtilitiesBusiness Services - Other,NaN,Experienced (Non-Manager)
a53be963aac0a938a50a4d4cf7bc3ca3,Company DescriptionProSource—a total office so...,Help Desk Support Engineer Job in West Chester,Full Time,"West Chester, OH",All,NaN,Experienced (Non-Manager)
1d37a888ca65fd919e459147a4c33457,"About Us: Viox Services, a wholly owned subsid...",Custodian Lead Job in Cincinnati,Full Time Employee,Location:,Real Estate/Property Management,NaN,Installation/Maintenance/Repair
7502ee8f0d324f86334c531fa8bcf663,RESPONSIBILITIES: ...,Accountant Job in Cincinnati,Full Time,"Cincinnati, OH 45249",Healthcare Services,NaN,Entry Level
9796e104240789dd33cc436f6c383892,"Full-Time Amber Park Cincinnati, OH 3801 East ...",Licensed Practical Nurse LPN Job in Cincinnati,Full Time Employee,"Cincinnati, OH 45236",Healthcare Services Other/Not Classified,NaN,Medical/Health
abd9ad3e0ec3c934b5a59f3776012865,What The Job Is AboutSales Support Representat...,Immediate Customer Service Position Job in Cin...,Part Time,"Cincinnati, OH 45202",All,NaN,Entry Level
419a3714be2b30a10f628de207d041de,Luxury homebuilder in Cincinnati seeking multi...,Accountant Job in Cincinnati,Full Time,"Cincinnati, OH 45236",Construction - Residential & Commercial/Office,"45,000.00 - 60,000.00 $ /year",Manager (Manager/Supervisor of Staff)


 In the last shape command we got 20000 rows and now after dropping na values of some columns we get 11847 rows, the only column that remain with na vlues is salary with 83 % of null values for future propouses

In [75]:
shape_ = df.shape
df.isnull().sum()/shape_[0]*100

job_description     0.000000
job_title           0.000000
job_type            0.000000
location            0.000000
organization        0.000000
salary             83.801806
sector              0.000000
dtype: float64


## In the time we are working with the dataset , we will be creating cleanse on Content Dirty and Structure Drity meanwhile eather of those is nedded

## Salary 


Salary column has a lot information that can be useful but we have to repair the content and structure of the information 
 
 - This column has a range between two values that we have to split
 * Lets split them
 we have to erase son characteres like '-' and '$'


In [76]:
df['salary'].unique()
# We can see that thera are numbers, '-', strings ,'$' 

array([nan, '9.00 - 13.00 $ /hour', '80,000.00 - 95,000.00 $ /year', ...,
       '55,000.00 - 60,000.00 $ /yearFull Benefits Package, Life Insurance, 401K, Relocation Support',
       '75,000.00 - 85,000.00 $ /yearHighly Competitive Base Salary PLUS Lucrative Bonus Plan in a Highly Diverse, Dynamic, Successful Company! Performance Based Upward Mobility Assured.\u200b Do Not Hesitate, Apply Today and Grow with this Nationally Present Award Winning Restaurant Group!',
       '25.00 - 28.00 $ /hour'], dtype=object)

In [77]:
#--------- SD -----------
df_sal_split = df['salary']
df_sal_split.tail(10)

# In what spliting is about I prefer to create a new subset of the data that is goin to be manipulated

uniq_id
8e9f8638556bc1fd671bb99f4f01ac4d    75,000.00 - 85,000.00 $ /yearHighly Competitiv...
ebce61a714f4dd7d15b0263fab42751e                                                  NaN
8a36252a31d7b06e901be0596bb6501a                                                  NaN
a53be963aac0a938a50a4d4cf7bc3ca3                                                  NaN
1d37a888ca65fd919e459147a4c33457                                                  NaN
7502ee8f0d324f86334c531fa8bcf663                                                  NaN
9796e104240789dd33cc436f6c383892                                                  NaN
abd9ad3e0ec3c934b5a59f3776012865                                                  NaN
419a3714be2b30a10f628de207d041de                        45,000.00 - 60,000.00 $ /year
40161cf61c283af9dc2b0a62947a5f1b                                25.00 - 28.00 $ /hour
Name: salary, dtype: object

In [78]:
df_sal_split= df_sal_split.str.split('/',expand=True)
df_sal_split
# We have to split the salary column and expand the results 


,0,1,2,3,4,5
uniq_id,,,,,,
58435fcab804439efdcaa7ecca0fd783,NaN,NaN,NaN,NaN,NaN,NaN
64d0272dc8496abfd9523a8df63c184c,NaN,NaN,NaN,NaN,NaN,NaN
1e2637cb5f7a2c4615a99a26c0566c66,NaN,NaN,NaN,NaN,NaN,NaN
a6a2b5e825b8ce1c3b517adb2497c5ed,NaN,NaN,NaN,NaN,NaN,NaN
2f8bdf60db4d85627ab8f040e67aa78d,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
7502ee8f0d324f86334c531fa8bcf663,NaN,NaN,NaN,NaN,NaN,NaN
9796e104240789dd33cc436f6c383892,NaN,NaN,NaN,NaN,NaN,NaN
abd9ad3e0ec3c934b5a59f3776012865,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
df_sal_split[0].unique()
# There are some characteres that replace the range of salary because those companies don't want to say at first the payment

array([nan, '9.00 - 13.00 $ ', '80,000.00 - 95,000.00 $ ',
       '70,000.00 - 100,000.00 $ ', '68,000.00 - 72,000.00 $ ',
       '58,000.00 - 65,000.00 $ ', 'Up to $32000.00',
       'Salary, plus commission', '45,000.00 - 100,000.00 $ ',
       '40,000.00 - 50,000.00 $ ', '13.75 - 16.75 $ ',
       '35,000.00 - 45,000.00 $ ',
       'bonus, 401K matching, medical, vacation',
       '31,000.00 - 33,000.00 $ ', '17.00 - 22.00 $ ',
       '56,000.00 - 64,000.00 $ ', '45.00 - 50.00 $ ',
       '75,000.00 - 130,000.00 $ ', 'Up to $45000.00',
       '0.00 - 85,000.00 $ ', 'Negotiable based on experience',
       '60,000.00 - 110,000.00 $ ', 'Competitive Wages',
       '50,000.00 - 100,000.00 $ ',
       'Burg Simpson offers excellent benefits and compensation commensurate with experience.',
       '$40,000.00+ ', 'Excellent compensation and benefits',
       '69,000.00 - 101,000.00 $ ', '15.00 - 19.00 $ ',
       '15.00 - 21.00 $ ', '13.00 - 16.00 $ ', '25,000.00 - 57,000.00 $ ',
       '1

In [80]:
df_sal_split[1].unique()
# These are the aditional conditions and benefits that one person can obtain into his contract

array([nan, 'hour', 'year', None, 'yearBonus, Benefits, 401k',
       'yearsalary', 'hourYear End Bonus',
       'yearHighly Competitive Base Salary Plus Lucrative Bonus Plan, Benefit Package, in a Highly Diverse, Very Successful Company! Performance Based Upward Mobility Assured.\u200b Do Not Hesitate, Apply Today.\u200b Grow with this Nationally Present, Dynamic Restaurant Group',
       'yearGenerous Commission plan', 'hourPlus benefits',
       'hourBenefits Package',
       'yearBase + Uncapped Commissions + Benefits',
       'hourBenefits + Annual Bonus', 'week',
       'Dental Benefits; 401k Employer Match',
       'yearbase salary plus bonus', 'year+ profit sharing',
       'yearpackage', 'yearPACKAGE', 'yearbonus, tips and comp time',
       'year5', 'hourgenerous benefit package',
       'year$8,000 Recruitment Incentive Pay may be available.',
       'yearPTO, 401K',
       'hourBenefits (401K, Health Insurance) and bonuses are potentials',
       'yearHealth Benefits', 'yea

In [81]:
df_sal_split[2].unique()
# FroM here  we can obtein some benefits perhaps we can join the column one to four

array([nan, None, 'life insurance, 401K', 'Bonus', '401K',
       ' monthly performance bonus & commission', ' Incentives',
       'paid time off', 'Outlet', ' Bonus Structure', 'dental',
       'experience', 'Dental'], dtype=object)

In [82]:
df_sal_split['conditions'] = df_sal_split[1].astype(str)+' '+df_sal_split[2].astype(str)+' '+df_sal_split[3].astype(str)+' '+df_sal_split[4].astype(str)+' '+df_sal_split[5].astype(str)
df_sal_split.tail(10)

# We join columns splited before in just one to manage the strings of the salaries conditions, At this momento I don't know if is going usefull but in case it doesn't 
# you can always drop it, I just don´t want to losse information that migh be useful

,0,1,2,3,4,5,conditions
uniq_id,,,,,,,
8e9f8638556bc1fd671bb99f4f01ac4d,"75,000.00 - 85,000.00 $",yearHighly Competitive Base Salary PLUS Lucrat...,None,None,None,None,yearHighly Competitive Base Salary PLUS Lucrat...
ebce61a714f4dd7d15b0263fab42751e,NaN,NaN,NaN,NaN,NaN,NaN,nan nan nan nan nan
8a36252a31d7b06e901be0596bb6501a,NaN,NaN,NaN,NaN,NaN,NaN,nan nan nan nan nan
a53be963aac0a938a50a4d4cf7bc3ca3,NaN,NaN,NaN,NaN,NaN,NaN,nan nan nan nan nan
1d37a888ca65fd919e459147a4c33457,NaN,NaN,NaN,NaN,NaN,NaN,nan nan nan nan nan
7502ee8f0d324f86334c531fa8bcf663,NaN,NaN,NaN,NaN,NaN,NaN,nan nan nan nan nan
9796e104240789dd33cc436f6c383892,NaN,NaN,NaN,NaN,NaN,NaN,nan nan nan nan nan
abd9ad3e0ec3c934b5a59f3776012865,NaN,NaN,NaN,NaN,NaN,NaN,nan nan nan nan nan
419a3714be2b30a10f628de207d041de,"45,000.00 - 60,000.00 $",year,None,None,None,None,year None None None None


In [83]:
df_sal_split_= df_sal_split['conditions']
df_sal_split_

#Now lets take just the column that collects the whole  information of conditions and benefits gaven by the companies, here nan replace empty spaces, we will clean this mess later

uniq_id
58435fcab804439efdcaa7ecca0fd783         nan nan nan nan nan
64d0272dc8496abfd9523a8df63c184c         nan nan nan nan nan
1e2637cb5f7a2c4615a99a26c0566c66         nan nan nan nan nan
a6a2b5e825b8ce1c3b517adb2497c5ed         nan nan nan nan nan
2f8bdf60db4d85627ab8f040e67aa78d         nan nan nan nan nan
                                              ...           
7502ee8f0d324f86334c531fa8bcf663         nan nan nan nan nan
9796e104240789dd33cc436f6c383892         nan nan nan nan nan
abd9ad3e0ec3c934b5a59f3776012865         nan nan nan nan nan
419a3714be2b30a10f628de207d041de    year None None None None
40161cf61c283af9dc2b0a62947a5f1b    hour None None None None
Name: conditions, Length: 11847, dtype: object

In [84]:
df_sal_split_.shape

(11847,)

In [85]:
df_sal_range= df_sal_split[0].str.split('-',expand=True)
df_sal_range
#  Now we are gonna focus on column cero with the ranges of salary an later on we are going back to the condition column
# Here we create a split that give us agaon 4 columns but

,0,1,2,3
uniq_id,,,,
58435fcab804439efdcaa7ecca0fd783,NaN,NaN,NaN,NaN
64d0272dc8496abfd9523a8df63c184c,NaN,NaN,NaN,NaN
1e2637cb5f7a2c4615a99a26c0566c66,NaN,NaN,NaN,NaN
a6a2b5e825b8ce1c3b517adb2497c5ed,NaN,NaN,NaN,NaN
2f8bdf60db4d85627ab8f040e67aa78d,NaN,NaN,NaN,NaN
...,...,...,...,...
7502ee8f0d324f86334c531fa8bcf663,NaN,NaN,NaN,NaN
9796e104240789dd33cc436f6c383892,NaN,NaN,NaN,NaN
abd9ad3e0ec3c934b5a59f3776012865,NaN,NaN,NaN,NaN


In [86]:
df_sal_range[0].unique()
# Here we manage to get the initial salary but it is messy with information of companies that don't like to say the salary for the possition

array([nan, '9.00 ', '80,000.00 ', '70,000.00 ', '68,000.00 ',
       '58,000.00 ', 'Up to $32000.00', 'Salary, plus commission',
       '45,000.00 ', '40,000.00 ', '13.75 ', '35,000.00 ',
       'bonus, 401K matching, medical, vacation', '31,000.00 ', '17.00 ',
       '56,000.00 ', '45.00 ', '75,000.00 ', 'Up to $45000.00', '0.00 ',
       'Negotiable based on experience', '60,000.00 ',
       'Competitive Wages', '50,000.00 ',
       'Burg Simpson offers excellent benefits and compensation commensurate with experience.',
       '$40,000.00+ ', 'Excellent compensation and benefits',
       '69,000.00 ', '15.00 ', '13.00 ', '25,000.00 ', '85,000.00 ',
       '10.00 ', '12.00 ', '11.85 ', '14.00 ',
       'Salary based on experience.', '$10.87+ ', '$12.00+ ', '30.93 ',
       '45,000.00+ ', '20.00 ', '13.45 ', '100,000.00 ', '55.00 ',
       '11.00 ', '16.00 ', '$16.40+ ', '90,000.00 ', '55,000.00 ',
       '$120,000.00+ ', '26.92 ', '$13.50+ ', '9.50 ', '19.50 ',
       '54,058.00 ', '

In [87]:
df_sal_range[1].unique()
# In column 1 we find the final salary for the possition that was offered
# There is alsa a character '$' that is no useful in this column

array([nan, ' 13.00 $ ', ' 95,000.00 $ ', ' 100,000.00 $ ',
       ' 72,000.00 $ ', ' 65,000.00 $ ', None, ' 50,000.00 $ ',
       ' 16.75 $ ', ' 45,000.00 $ ', ' 33,000.00 $ ', ' 22.00 $ ',
       ' 64,000.00 $ ', ' 50.00 $ ', ' 130,000.00 $ ', ' 85,000.00 $ ',
       ' 110,000.00 $ ', ' 101,000.00 $ ', ' 19.00 $ ', ' 21.00 $ ',
       ' 16.00 $ ', ' 57,000.00 $ ', ' 17.00 $ ', ' 80,000.00 $ ',
       ' 60,000.00 $ ', ' 11.00 $ ', ' 90,000.00 $ ', ' 14.00 $ ',
       ' 11.85 $ ', ' 30.93 $ ', ' 23.00 $ ', ' 12.25 $ ', ' 13.45 $ ',
       ' 130,000.00 ', ' 50,000.00 ', ' 65.00 $ ', ' 17.00 ',
       ' 85,000.00 ', ' 15.00 $ ', ' 105,000.00 ', ' 26.92 $ ',
       ' 21.90 $ ', ' 75,000.00 $ ', ' 18.00 $ ', ' 100,000.00 ',
       ' 19.50 $ ', ' 79,174.00 $ ', ' 19.00 ', ' 55,000.00 $ ',
       'On Bonus', ' 66,000.00 $ ', ' 15.00 ', ' 35.00 $ ', ' 120.00 $ ',
       ' 200,000.00 $ ', ' 20.00 $ ', ' 100.00 $ ', ' 75.00 $ ',
       ' 80.00 $ ', ' 220,000.00 $ ', ' 32.00 $ ', ' 12.00 $ ',
  

In [88]:
df_sal_range_ = df_sal_range.loc[:,0:1].rename(columns={0:'initial_sal',1:'final_sal'})
df_sal_range_
# What we do here is just taking the columns we consider important for the dataset to be analyse later regarding salary

,initial_sal,final_sal
uniq_id,,
58435fcab804439efdcaa7ecca0fd783,NaN,NaN
64d0272dc8496abfd9523a8df63c184c,NaN,NaN
1e2637cb5f7a2c4615a99a26c0566c66,NaN,NaN
a6a2b5e825b8ce1c3b517adb2497c5ed,NaN,NaN
2f8bdf60db4d85627ab8f040e67aa78d,NaN,NaN
...,...,...
7502ee8f0d324f86334c531fa8bcf663,NaN,NaN
9796e104240789dd33cc436f6c383892,NaN,NaN
abd9ad3e0ec3c934b5a59f3776012865,NaN,NaN


In [89]:
df2 = pd.DataFrame(df_sal_split_)
df3 = pd.DataFrame(df_sal_range_)

# We have to give pandas formart to the series of objects that we create before

df_salaries = df2.join(df3, how='left')
df_salaries
#No we df_salaries the two past datasets that split in order to clean the structure of this salary column
#As we can see we create:
# sal_conditions = conditions of the salary
# sal_from = intitial salary
# sal_to = final salary

# But..... still there is something wrong about this chunck of dataset in final_sal column, Can you see it? I'll give you a clue '$'

,conditions,initial_sal,final_sal
uniq_id,,,
58435fcab804439efdcaa7ecca0fd783,nan nan nan nan nan,NaN,NaN
64d0272dc8496abfd9523a8df63c184c,nan nan nan nan nan,NaN,NaN
1e2637cb5f7a2c4615a99a26c0566c66,nan nan nan nan nan,NaN,NaN
a6a2b5e825b8ce1c3b517adb2497c5ed,nan nan nan nan nan,NaN,NaN
2f8bdf60db4d85627ab8f040e67aa78d,nan nan nan nan nan,NaN,NaN
...,...,...,...
7502ee8f0d324f86334c531fa8bcf663,nan nan nan nan nan,NaN,NaN
9796e104240789dd33cc436f6c383892,nan nan nan nan nan,NaN,NaN
abd9ad3e0ec3c934b5a59f3776012865,nan nan nan nan nan,NaN,NaN


In [90]:
df_salaries

,conditions,initial_sal,final_sal
uniq_id,,,
58435fcab804439efdcaa7ecca0fd783,nan nan nan nan nan,NaN,NaN
64d0272dc8496abfd9523a8df63c184c,nan nan nan nan nan,NaN,NaN
1e2637cb5f7a2c4615a99a26c0566c66,nan nan nan nan nan,NaN,NaN
a6a2b5e825b8ce1c3b517adb2497c5ed,nan nan nan nan nan,NaN,NaN
2f8bdf60db4d85627ab8f040e67aa78d,nan nan nan nan nan,NaN,NaN
...,...,...,...
7502ee8f0d324f86334c531fa8bcf663,nan nan nan nan nan,NaN,NaN
9796e104240789dd33cc436f6c383892,nan nan nan nan nan,NaN,NaN
abd9ad3e0ec3c934b5a59f3776012865,nan nan nan nan nan,NaN,NaN


### Salary clean data
We just clean the structure of the column salary now we are going to clear the data itself

In [91]:
 #------------CD-----------------
 
df_salaries['initial_sal_'] = df_salaries['initial_sal'].str.extract('(\d.+)')
df_salaries
# In this ocation we are gonna  erase the string of the column 'initial data' so we can use just numbers

,conditions,initial_sal,final_sal,initial_sal_
uniq_id,,,,
58435fcab804439efdcaa7ecca0fd783,nan nan nan nan nan,NaN,NaN,NaN
64d0272dc8496abfd9523a8df63c184c,nan nan nan nan nan,NaN,NaN,NaN
1e2637cb5f7a2c4615a99a26c0566c66,nan nan nan nan nan,NaN,NaN,NaN
a6a2b5e825b8ce1c3b517adb2497c5ed,nan nan nan nan nan,NaN,NaN,NaN
2f8bdf60db4d85627ab8f040e67aa78d,nan nan nan nan nan,NaN,NaN,NaN
...,...,...,...,...
7502ee8f0d324f86334c531fa8bcf663,nan nan nan nan nan,NaN,NaN,NaN
9796e104240789dd33cc436f6c383892,nan nan nan nan nan,NaN,NaN,NaN
abd9ad3e0ec3c934b5a59f3776012865,nan nan nan nan nan,NaN,NaN,NaN


In [92]:
df_salaries=df_salaries.drop(['initial_sal'], axis = 1)
df_salaries

,conditions,final_sal,initial_sal_
uniq_id,,,
58435fcab804439efdcaa7ecca0fd783,nan nan nan nan nan,NaN,NaN
64d0272dc8496abfd9523a8df63c184c,nan nan nan nan nan,NaN,NaN
1e2637cb5f7a2c4615a99a26c0566c66,nan nan nan nan nan,NaN,NaN
a6a2b5e825b8ce1c3b517adb2497c5ed,nan nan nan nan nan,NaN,NaN
2f8bdf60db4d85627ab8f040e67aa78d,nan nan nan nan nan,NaN,NaN
...,...,...,...
7502ee8f0d324f86334c531fa8bcf663,nan nan nan nan nan,NaN,NaN
9796e104240789dd33cc436f6c383892,nan nan nan nan nan,NaN,NaN
abd9ad3e0ec3c934b5a59f3776012865,nan nan nan nan nan,NaN,NaN


In [93]:
#---------------CD-----------------------
df_salaries['final_sal'] = df_salaries['final_sal'].str.replace('$', '')
df_salaries

#Lets now delete the '$' Character form column 'final_sal'


C:\Users\yohan\AppData\Local\Temp\ipykernel_3352\2682389010.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_salaries['final_sal'] = df_salaries['final_sal'].str.replace('$', '')


,conditions,final_sal,initial_sal_
uniq_id,,,
58435fcab804439efdcaa7ecca0fd783,nan nan nan nan nan,NaN,NaN
64d0272dc8496abfd9523a8df63c184c,nan nan nan nan nan,NaN,NaN
1e2637cb5f7a2c4615a99a26c0566c66,nan nan nan nan nan,NaN,NaN
a6a2b5e825b8ce1c3b517adb2497c5ed,nan nan nan nan nan,NaN,NaN
2f8bdf60db4d85627ab8f040e67aa78d,nan nan nan nan nan,NaN,NaN
...,...,...,...
7502ee8f0d324f86334c531fa8bcf663,nan nan nan nan nan,NaN,NaN
9796e104240789dd33cc436f6c383892,nan nan nan nan nan,NaN,NaN
abd9ad3e0ec3c934b5a59f3776012865,nan nan nan nan nan,NaN,NaN


In [94]:
df_ = df_salaries.join(df, how='left').drop(['salary'], axis = 1)
df_
# We have the dataset with Salary c0lumn CD and SD 
# We will retake this c0lumn later to create a mean value that could or not be useful for imputation

,conditions,final_sal,initial_sal_,job_description,job_title,job_type,location,organization,sector
uniq_id,,,,,,,,,
58435fcab804439efdcaa7ecca0fd783,nan nan nan nan nan,NaN,NaN,Why Join Altec? If you’re considering a career...,Engineer - Quality Job in Dixon,Full Time,"Dixon, CA",Altec Industries,Experienced (Non-Manager)
64d0272dc8496abfd9523a8df63c184c,nan nan nan nan nan,NaN,NaN,Position ID# 76162 # Positions 1 State CT C...,Shift Supervisor - Part-Time Job in Camphill,Full Time Employee,"Camphill, PA",Retail,Project/Program Management
1e2637cb5f7a2c4615a99a26c0566c66,nan nan nan nan nan,NaN,NaN,Job Description Job #: 720298Apex Systems has...,Construction PM - Charlottesville Job in Charl...,Full Time Employee,"Charlottesville, VA",Computer/IT Services,Experienced (Non-Manager)
a6a2b5e825b8ce1c3b517adb2497c5ed,nan nan nan nan nan,NaN,NaN,"Part-Time, 4:30 pm - 9:30 pm, Mon - Fri Brookd...",Housekeeper Job in Austin,Part Time Employee,"Austin, TX 78746",Hotels and Lodging Personal and Household Serv...,Customer Support/Client Care
2f8bdf60db4d85627ab8f040e67aa78d,nan nan nan nan nan,NaN,NaN,Aflac Insurance Sales Agent While a career in ...,Aflac Insurance Sales Agent Job in Berryville,Full Time,"Berryville, VA 22611",Insurance,Customer Support/Client Care
...,...,...,...,...,...,...,...,...,...
7502ee8f0d324f86334c531fa8bcf663,nan nan nan nan nan,NaN,NaN,RESPONSIBILITIES: ...,Accountant Job in Cincinnati,Full Time,"Cincinnati, OH 45249",Healthcare Services,Entry Level
9796e104240789dd33cc436f6c383892,nan nan nan nan nan,NaN,NaN,"Full-Time Amber Park Cincinnati, OH 3801 East ...",Licensed Practical Nurse LPN Job in Cincinnati,Full Time Employee,"Cincinnati, OH 45236",Healthcare Services Other/Not Classified,Medical/Health
abd9ad3e0ec3c934b5a59f3776012865,nan nan nan nan nan,NaN,NaN,What The Job Is AboutSales Support Representat...,Immediate Customer Service Position Job in Cin...,Part Time,"Cincinnati, OH 45202",All,Entry Level


In [95]:
shape=df_.shape
df_.isnull().sum()/shape[0]*100

conditions          0.000000
final_sal          87.802819
initial_sal_       86.021778
job_description     0.000000
job_title           0.000000
job_type            0.000000
location            0.000000
organization        0.000000
sector              0.000000
dtype: float64

In [96]:
#----------- SD-----------------
#from this part we are going to work with what could be crusial to predict the salary of the missing rows, this informarion was obtain from Salary column
#and now are the conditions of the salary

df_noempty = df_[~df_['initial_sal_'].isna()]
df_noempty_cond = df_noempty['conditions']
df_noempty_cond.tail(5)

# First we are goin to select salarys column that is not empty, as we can see there is information of the period year month , week

uniq_id
4e2b0a3e9fe5f8721f6ab4692823d9a9    week7-9% commisson depending on sales volume N...
ac3c743eb7612d90bfe80a3a30d3d8be    yearFull Benefits Package, Life Insurance, 401...
8e9f8638556bc1fd671bb99f4f01ac4d    yearHighly Competitive Base Salary PLUS Lucrat...
419a3714be2b30a10f628de207d041de                             year None None None None
40161cf61c283af9dc2b0a62947a5f1b                             hour None None None None
Name: conditions, dtype: object

In [97]:
df_noempty_cond_2 = df_noempty_cond.replace('None', '', regex=True)
df_noempty_cond_2
# The goal is to transponde values to columns

uniq_id
b43c077756d5a326c4854e1399fd2464                                             hour    
d8491fcefe14d1398de419984dccf427                                             year    
779bb4c9bf038b7fb775134736d36fd4                                             year    
ceb44cca7cd280adcb0c84c20f3c6c21                                             year    
eea9b50afc4fece9f9d6ff0dbf659784                                             year    
                                                          ...                        
4e2b0a3e9fe5f8721f6ab4692823d9a9     week7-9% commisson depending on sales volume    
ac3c743eb7612d90bfe80a3a30d3d8be    yearFull Benefits Package, Life Insurance, 401...
8e9f8638556bc1fd671bb99f4f01ac4d    yearHighly Competitive Base Salary PLUS Lucrat...
419a3714be2b30a10f628de207d041de                                             year    
40161cf61c283af9dc2b0a62947a5f1b                                             hour    
Name: conditions, Length: 1656, dtype: object

In [98]:
#-----------CD------------------
df_noempty_cond_2= df_noempty_cond_2.str.lower()
df_noempty_cond_2_ = df_noempty_cond_2.to_frame()
df_noempty_cond_2_ 
#It is always better to lower characters in order to have uniformity queries work better an easy when data is uniform

,conditions
uniq_id,
b43c077756d5a326c4854e1399fd2464,hour
d8491fcefe14d1398de419984dccf427,year
779bb4c9bf038b7fb775134736d36fd4,year
ceb44cca7cd280adcb0c84c20f3c6c21,year
eea9b50afc4fece9f9d6ff0dbf659784,year
...,...
4e2b0a3e9fe5f8721f6ab4692823d9a9,week7-9% commisson depending on sales volume
ac3c743eb7612d90bfe80a3a30d3d8be,"yearfull benefits package, life insurance, 401..."
8e9f8638556bc1fd671bb99f4f01ac4d,yearhighly competitive base salary plus lucrat...


In [99]:
df_noempty_cond_split = df_noempty_cond_2_['conditions'].str.split(' ',n=1 ,expand=True)
df_noempty_cond_3 = df_noempty_cond_split.replace([None], '', regex=True)
df_noempty_cond_3.tail(5)

#Fron the column 'condition' lets split using spaces to get the perios of time that might make some correlation with salary column

,0,1
uniq_id,,
4e2b0a3e9fe5f8721f6ab4692823d9a9,week7-9%,commisson depending on sales volume
ac3c743eb7612d90bfe80a3a30d3d8be,yearfull,"benefits package, life insurance, 401k, reloca..."
8e9f8638556bc1fd671bb99f4f01ac4d,yearhighly,competitive base salary plus lucrative bonus p...
419a3714be2b30a10f628de207d041de,year,
40161cf61c283af9dc2b0a62947a5f1b,hour,


In [100]:
df_noempty_cond_3['period'] = df_noempty_cond_3[0].astype(str).str[0:4]  # Select the period of time
df_noempty_cond_3['period_cond'] = df_noempty_cond_3[0].astype(str).str[4:-1] # Select some conditions of the period 
df_noempty_cond_3['period'].unique()
# in period we have three undesirable characters "hr",2%" and "per" the are just 4 rows, s to not

array(['hour', 'year', '', 'week', 'mont', '2%,', 'per', 'hr'],
      dtype=object)

In [101]:
df_noempty_cond_3_ = df_noempty_cond_3[df_noempty_cond_3['period']!= '2%,']
df_noempty_cond_3_ = df_noempty_cond_3_[df_noempty_cond_3['period']!= 'hr']
df_noempty_cond_3_ = df_noempty_cond_3_[df_noempty_cond_3['period']!= 'per']

C:\Users\yohan\AppData\Local\Temp\ipykernel_3352\716745779.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_noempty_cond_3_ = df_noempty_cond_3_[df_noempty_cond_3['period']!= 'hr']
C:\Users\yohan\AppData\Local\Temp\ipykernel_3352\716745779.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_noempty_cond_3_ = df_noempty_cond_3_[df_noempty_cond_3['period']!= 'per']


In [102]:
df_noempty_period = df_noempty_cond_3_.drop(0, axis = 1) # Drop column replaced
df_noempty_period_ = df_noempty_period.rename(columns= {1: 'benefits'}) # Rename column
df_noempty_period_['benefits'] = df_noempty_period_['benefits'].str.strip() # Seperate information that we coul benefits of worwing in a company
df_noempty_period_['period'].unique()

# It was not easy to obtaoin the period of work but some how I manage to extract it, lucke me  hour, year and week have the same amount of charaters

array(['hour', 'year', '', 'week', 'mont'], dtype=object)

In [103]:
df_2 = df_.join(df_noempty_period_, how='left').drop(columns='conditions')
df_2.head(5)
# The join() function performs a left join by default, nevertheless I use how=left, so each of the indexes in the first DataFrame are kept
# We are going to repeat this procedure several times.

,final_sal,initial_sal_,job_description,job_title,job_type,location,organization,sector,benefits,period,period_cond
uniq_id,,,,,,,,,,,
58435fcab804439efdcaa7ecca0fd783,NaN,NaN,Why Join Altec? If you’re considering a career...,Engineer - Quality Job in Dixon,Full Time,"Dixon, CA",Altec Industries,Experienced (Non-Manager),NaN,NaN,NaN
64d0272dc8496abfd9523a8df63c184c,NaN,NaN,Position ID# 76162 # Positions 1 State CT C...,Shift Supervisor - Part-Time Job in Camphill,Full Time Employee,"Camphill, PA",Retail,Project/Program Management,NaN,NaN,NaN
1e2637cb5f7a2c4615a99a26c0566c66,NaN,NaN,Job Description Job #: 720298Apex Systems has...,Construction PM - Charlottesville Job in Charl...,Full Time Employee,"Charlottesville, VA",Computer/IT Services,Experienced (Non-Manager),NaN,NaN,NaN
a6a2b5e825b8ce1c3b517adb2497c5ed,NaN,NaN,"Part-Time, 4:30 pm - 9:30 pm, Mon - Fri Brookd...",Housekeeper Job in Austin,Part Time Employee,"Austin, TX 78746",Hotels and Lodging Personal and Household Serv...,Customer Support/Client Care,NaN,NaN,NaN
2f8bdf60db4d85627ab8f040e67aa78d,NaN,NaN,Aflac Insurance Sales Agent While a career in ...,Aflac Insurance Sales Agent Job in Berryville,Full Time,"Berryville, VA 22611",Insurance,Customer Support/Client Care,NaN,NaN,NaN


## job_title


Job title has information about the position but also has information about job_type an location that already exist in other columns so the only information in this column that reallly maters is the offer job.

In [104]:
df_job_title = df_['job_title']
df_job_title_= df_job_title.to_frame()
df_job_title_

,job_title
uniq_id,
58435fcab804439efdcaa7ecca0fd783,Engineer - Quality Job in Dixon
64d0272dc8496abfd9523a8df63c184c,Shift Supervisor - Part-Time Job in Camphill
1e2637cb5f7a2c4615a99a26c0566c66,Construction PM - Charlottesville Job in Charl...
a6a2b5e825b8ce1c3b517adb2497c5ed,Housekeeper Job in Austin
2f8bdf60db4d85627ab8f040e67aa78d,Aflac Insurance Sales Agent Job in Berryville
...,...
7502ee8f0d324f86334c531fa8bcf663,Accountant Job in Cincinnati
9796e104240789dd33cc436f6c383892,Licensed Practical Nurse LPN Job in Cincinnati
abd9ad3e0ec3c934b5a59f3776012865,Immediate Customer Service Position Job in Cin...


In [105]:
df_job_title_in = df_job_title_['job_title'].str.rsplit('Job in', expand= True)
df_job_title_in.head(25)

,0,1
uniq_id,,
58435fcab804439efdcaa7ecca0fd783,Engineer - Quality,Dixon
64d0272dc8496abfd9523a8df63c184c,Shift Supervisor - Part-Time,Camphill
1e2637cb5f7a2c4615a99a26c0566c66,Construction PM - Charlottesville,Charlottesville
a6a2b5e825b8ce1c3b517adb2497c5ed,Housekeeper,Austin
2f8bdf60db4d85627ab8f040e67aa78d,Aflac Insurance Sales Agent,Berryville
3a2adeacbf35985a508f630661eebb43,Sales Associate,Columbus
b43c077756d5a326c4854e1399fd2464,Primrose Private Preschool Teacher,Houston
d8491fcefe14d1398de419984dccf427,Superintendent,Houston
8d8f39de4a07929a92c2e6dd7a72a0af,Pest Control Technician – Field Service Rep (E...,Denver


In [106]:
df_job_title_in = df_job_title_in[0].str.split(' -', expand= True)
df_job_title_in= df_job_title_in[0].str.split('- ', expand= True)
df_job_title_in= df_job_title_in[0].str.split(' – ', expand= True)
df_job_title_in = df_job_title_in[0].str.lower().to_frame()

df_job_title_in.head(20)

,0
uniq_id,
58435fcab804439efdcaa7ecca0fd783,engineer
64d0272dc8496abfd9523a8df63c184c,shift supervisor
1e2637cb5f7a2c4615a99a26c0566c66,construction pm
a6a2b5e825b8ce1c3b517adb2497c5ed,housekeeper
2f8bdf60db4d85627ab8f040e67aa78d,aflac insurance sales agent
3a2adeacbf35985a508f630661eebb43,sales associate
b43c077756d5a326c4854e1399fd2464,primrose private preschool teacher
d8491fcefe14d1398de419984dccf427,superintendent
8d8f39de4a07929a92c2e6dd7a72a0af,pest control technician


In [107]:
df_job_title_in_ = df_job_title_in[0].str.replace('w/','|').str.replace('/','|').str.replace('&','|').str.replace('.','')\
    .str.replace('+', '|').str.replace('iii', '|').str.replace('ii', '').str.replace('*', '|').str.replace('$', '|').str.replace(':', '|')\
        .str.replace('(', '|').str.replace('#', '|').str.replace('[', '|').str.replace('--', '|').str.replace('t I', 't').str.replace('{', '|') \
            .str.replace('?', '|').str.replace(' — ', '|').str.replace('!', '|').str.replace('-A', '|').str.replace('_', '|')\
                .str.replace('te-', 'te|').str.replace('zzz', '').str.replace('e I ', 'e').str.replace('senior', 'sr')\
                    .str.replace('assistant store manager', 'asst store mgr').str.replace('aflac', 'aflac insurance sales agent') \
                        .str.replace(' i ', '|').str.replace(' ii ', '|').str.replace('@', '|').str.replace(' u ', '|').str.replace('junior', 'jr')\
                            .str.replace('custodial lead', 'custodian').str.replace('director ', 'director|').str.replace('direct', 'director|')\
                                 .str.replace('sales aco ', 'sales|').str.replace('specialist part', 'specialist|').str.replace('executive assistant', 'executive assistant|')\
                                     .str.replace('experienced cdl-a driver', 'experienced cdl-a driver|').str.replace('facilities', 'facility')\
                                         .str.replace('field service engineer-', 'field service engineer|').str.replace('field supervisor ', 'field supervisor|').str.replace('finance–', 'finance|')\
                                             .str.replace('finance–', 'finance|').str.replace('front end web developer', 'front end developer').str.replace('front-end developer', 'front end developer')\
                                                 .str.replace('technician', 'tech').str.replace('field service tech', 'field tech').str.replace('full time ', '').str.replace('general labor', 'general laborer').str.replace('maintenance', 'maint').str.replace('mechanic c ', 'mechanic')\
                                                     .str.replace('general manager', 'general manager|').str.replace('government', 'government|').str.replace('helpdesk support', 'helpdesk support|').str.replace('physical therapist', 'physical therapist|')\
                                                         .str.replace('registered nurse', 'registered nurse|').str.replace('housekeeping supervisor', 'housekeeping supervisor|').str.replace('resource', 'resources|').str.replace('hvac installation', 'hvac installer')\
                                                            .str.replace('infection preventionist', 'infection preventionist|').str.replace('project manager', 'pm').str.replace('representative', 'rep').str.replace('inventory', 'inventory|').str.replace('janitorial', 'janitor|').str.replace(' job ', '')\
                                                                .str.replace('jde', 'java developer').str.replace('jr administrative assistant', 'jr administrative assistant|').str.replace('kitchen manager', 'kitchen manager|').str.replace('asistant', 'asistants').str.replace('laborer', 'laborers').str.replace('marketing communications', 'marketing communications|').str.replace('part time', '')\
                                                                    .str.replace('legal secretary', 'legal secretary|').str.replace('litigation associate', 'litigation associate|').str.replace('machine operator', 'machine operator|').str.replace('machinist', 'machinist|').str.replace('manufacturing associate', 'manufacturing associate|').str.replace('medical assistant', 'medical assistant|')\
                                                                      .str.replace('net web developer', 'net developer').str.replace('engineer l ', 'engineer').str.replace('now hiring', '').str.replace('amazing ', '').str.replace('manager', 'mgr').str.replace('nurse practition', 'nurse practition|').str.replace('nursing', 'nurses').str.replace('occupational therapist', 'occupational therapist|')\
                                                                        .str.replace('member services rep', 'member services rep|').str.replace('operations mgr', 'operations mgr|').str.replace('database administrator', 'dba').str.replace('clerk-', 'clerk').str.replace('per diem ', '').str.replace('pre-op rn', 'pre-op').str.replace('production mgr', 'production mgr|').str.replace('operators', 'operator')\
                                                                            .str.replace('production worker', 'production worker|').str.replace('project coordinator', 'project coordinator|').str.replace('project engineering', 'project engineering|').str.replace('manger ', 'mgr|').str.replace('assurance engineer', 'assurance engineer|').str.replace('control tech', 'control tech|').str.replace('quality engineer', 'quality engineer|')\
                                                                                .str.replace('needed', '').str.replace('recruitment', 'recruiting').str.replace('coordinator-stores', 'coordinator-stores|').str.replace('immediately', '').str.replace('sales mgr', 'sales mgr|').str.replace('respiratory therapist', 'respiratory therapist|').str.replace('management', 'mgr').str.replace('restaurant mgr', 'restaurant mgr|').str.replace('merchandiser', 'merchandiser|')\
                                                                                    .str.replace('sales reps', 'sales reps|').str.replace('scientist ', 'scientist|').str.replace('service tech i', 'service tech').str.replace('administrator', 'admin').str.replace(' and ', '|').str.replace('sports minded', 'sports minded|').str.replace('tech i', 'tech').str.replace('user experience', 'ui').str.replace('wisconsin ', '')\
                                                                                        .to_frame()

df_job_title_in1= df_job_title_in_[0].str.split('|', expand= True)

df_job_title_in2= df_job_title_in1[0].sort_values().to_frame()
df_job_title_in2

C:\Users\yohan\AppData\Local\Temp\ipykernel_3352\514502016.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_job_title_in_ = df_job_title_in[0].str.replace('w/','|').str.replace('/','|').str.replace('&','|').str.replace('.','')\
C:\Users\yohan\AppData\Local\Temp\ipykernel_3352\514502016.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  .str.replace('+', '|').str.replace('iii', '|').str.replace('ii', '').str.replace('*', '|').str.replace('$', '|').str.replace(':', '|')\
C:\Users\yohan\AppData\Local\Temp\ipykernel_3352\514502016.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expres

,0
uniq_id,
95438bfa85819e64da24e3c9b8fffd76,
71bfac3cb6a2762c131b66730662bcbe,
2c594aad12dc6ae51125bff47d6c6e9a,
2cd570b4eaec6cc5b9098c611ae82332,
7401f2524514dd4f074e34389c9b4233,
...,...
244e22b8fa50615e945a3af444d70d25,xrf tech
df4d042234bd4f290f3cdb15d7137305,yard worker
914a112b91a87854ac386b94b74b68cd,yardi analyst


In [108]:
df_job_title_in2.to_csv('df_lista_de_jobs.csv')

In [109]:
df_job_title_in2[0].value_counts()

shift supervisor                   258
registered nurse                   219
restaurant mgr                     129
director                           124
pm                                  98
                                  ... 
furniture delivery associate         1
funeral director                     1
funds transfer wire processor        1
functional automation tester         1
youth program mgr                    1
Name: 0, Length: 5604, dtype: int64

In [110]:
df_3 = df_2.join(df_job_title_in2, how= 'left').rename(columns={0:'job'}).drop(columns = ['job_title'])
df_3['job_type']= df_3['job_type'].replace('Employee', '', regex=True).replace(',','').str.lower()
df_3.head(5)


,final_sal,initial_sal_,job_description,job_type,location,organization,sector,benefits,period,period_cond,job
uniq_id,,,,,,,,,,,
58435fcab804439efdcaa7ecca0fd783,NaN,NaN,Why Join Altec? If you’re considering a career...,full time,"Dixon, CA",Altec Industries,Experienced (Non-Manager),NaN,NaN,NaN,engineer
64d0272dc8496abfd9523a8df63c184c,NaN,NaN,Position ID# 76162 # Positions 1 State CT C...,full time,"Camphill, PA",Retail,Project/Program Management,NaN,NaN,NaN,shift supervisor
1e2637cb5f7a2c4615a99a26c0566c66,NaN,NaN,Job Description Job #: 720298Apex Systems has...,full time,"Charlottesville, VA",Computer/IT Services,Experienced (Non-Manager),NaN,NaN,NaN,construction pm
a6a2b5e825b8ce1c3b517adb2497c5ed,NaN,NaN,"Part-Time, 4:30 pm - 9:30 pm, Mon - Fri Brookd...",part time,"Austin, TX 78746",Hotels and Lodging Personal and Household Serv...,Customer Support/Client Care,NaN,NaN,NaN,housekeeper
2f8bdf60db4d85627ab8f040e67aa78d,NaN,NaN,Aflac Insurance Sales Agent While a career in ...,full time,"Berryville, VA 22611",Insurance,Customer Support/Client Care,NaN,NaN,NaN,aflac insurance sales agent insurance sales ag...


## Job_type
### Here we just take the type of job and delete aditional information that might not be usefull

In [111]:
#------------SD---------------
df_4= df_3['job_type']
df_4 = df_4.str.split('/',expand=True)
df_4_ = df_4[0].str.split(',',expand=True)
df_4_[0] = df_4_[0].str.strip()
df_4_[0].unique()

#We clean job type makin to splits for characters "/" and "," and then stripping 


array(['full time', 'part time', 'full time temporary',
       'full time intern', '', 'temporary', 'per diem',
       'part time temporary', 'exempt', 'part time seasonal',
       'part time intern'], dtype=object)

In [112]:
df_5 = df_3.join(df_4_[0], how='left').drop(columns='job_type').rename(columns= {0:'job_type'})
df_5['job_type'] = df_5['job_type'].replace(',', '')

df_5.head()

,final_sal,initial_sal_,job_description,location,organization,sector,benefits,period,period_cond,job,job_type
uniq_id,,,,,,,,,,,
58435fcab804439efdcaa7ecca0fd783,NaN,NaN,Why Join Altec? If you’re considering a career...,"Dixon, CA",Altec Industries,Experienced (Non-Manager),NaN,NaN,NaN,engineer,full time
64d0272dc8496abfd9523a8df63c184c,NaN,NaN,Position ID# 76162 # Positions 1 State CT C...,"Camphill, PA",Retail,Project/Program Management,NaN,NaN,NaN,shift supervisor,full time
1e2637cb5f7a2c4615a99a26c0566c66,NaN,NaN,Job Description Job #: 720298Apex Systems has...,"Charlottesville, VA",Computer/IT Services,Experienced (Non-Manager),NaN,NaN,NaN,construction pm,full time
a6a2b5e825b8ce1c3b517adb2497c5ed,NaN,NaN,"Part-Time, 4:30 pm - 9:30 pm, Mon - Fri Brookd...","Austin, TX 78746",Hotels and Lodging Personal and Household Serv...,Customer Support/Client Care,NaN,NaN,NaN,housekeeper,part time
2f8bdf60db4d85627ab8f040e67aa78d,NaN,NaN,Aflac Insurance Sales Agent While a career in ...,"Berryville, VA 22611",Insurance,Customer Support/Client Care,NaN,NaN,NaN,aflac insurance sales agent insurance sales ag...,full time


## location
- This column has information about city, state and other kind that is not usefull
- In this oportunity I'm just interested on city and state

In [113]:
#------------SD---------------

df_6 = df_5['location']
df_6 = df_6.str.split(',', n=2 , expand=True)
df_6[['city','state']] = df_6[[0,1]]
df_6 = df_6.drop(columns=[0,1,2])
df_6['city'] = df_6['city'].str.lower()
df_6['state'] = df_6['state'].str.lower()
df_6
# keep cleaning struicture of the data an lower characteres

,city,state
uniq_id,,
58435fcab804439efdcaa7ecca0fd783,dixon,ca
64d0272dc8496abfd9523a8df63c184c,camphill,pa
1e2637cb5f7a2c4615a99a26c0566c66,charlottesville,va
a6a2b5e825b8ce1c3b517adb2497c5ed,austin,tx 78746
2f8bdf60db4d85627ab8f040e67aa78d,berryville,va 22611
...,...,...
7502ee8f0d324f86334c531fa8bcf663,cincinnati,oh 45249
9796e104240789dd33cc436f6c383892,cincinnati,oh 45236
abd9ad3e0ec3c934b5a59f3776012865,cincinnati,oh 45202


In [114]:
df_7 = df_5.join(df_6, how='left').drop(columns='location')
df_7.head(5)

,final_sal,initial_sal_,job_description,organization,sector,benefits,period,period_cond,job,job_type,city,state
uniq_id,,,,,,,,,,,,
58435fcab804439efdcaa7ecca0fd783,NaN,NaN,Why Join Altec? If you’re considering a career...,Altec Industries,Experienced (Non-Manager),NaN,NaN,NaN,engineer,full time,dixon,ca
64d0272dc8496abfd9523a8df63c184c,NaN,NaN,Position ID# 76162 # Positions 1 State CT C...,Retail,Project/Program Management,NaN,NaN,NaN,shift supervisor,full time,camphill,pa
1e2637cb5f7a2c4615a99a26c0566c66,NaN,NaN,Job Description Job #: 720298Apex Systems has...,Computer/IT Services,Experienced (Non-Manager),NaN,NaN,NaN,construction pm,full time,charlottesville,va
a6a2b5e825b8ce1c3b517adb2497c5ed,NaN,NaN,"Part-Time, 4:30 pm - 9:30 pm, Mon - Fri Brookd...",Hotels and Lodging Personal and Household Serv...,Customer Support/Client Care,NaN,NaN,NaN,housekeeper,part time,austin,tx 78746
2f8bdf60db4d85627ab8f040e67aa78d,NaN,NaN,Aflac Insurance Sales Agent While a career in ...,Insurance,Customer Support/Client Care,NaN,NaN,NaN,aflac insurance sales agent insurance sales ag...,full time,berryville,va 22611


## organization

This column has a lot of articles that are a problem at the momento of spliting since a lot of columns a created so the best way to avoid this is deleting this information from the string

In [115]:
##------------CD---------------
replacements = [('and', ''), ('&', ''),('or',''),('-',' '),('/',' '),(',',' ')]

for char, replacement in replacements:
    df_7['organization'] =  df_7['organization'].map(lambda x: x.replace(char,replacement))
df_7.head(5)

# we can use map() to apply a lambda function that removes articles from the beginning of the string to the end of the string.


,final_sal,initial_sal_,job_description,organization,sector,benefits,period,period_cond,job,job_type,city,state
uniq_id,,,,,,,,,,,,
58435fcab804439efdcaa7ecca0fd783,NaN,NaN,Why Join Altec? If you’re considering a career...,Altec Industries,Experienced (Non-Manager),NaN,NaN,NaN,engineer,full time,dixon,ca
64d0272dc8496abfd9523a8df63c184c,NaN,NaN,Position ID# 76162 # Positions 1 State CT C...,Retail,Project/Program Management,NaN,NaN,NaN,shift supervisor,full time,camphill,pa
1e2637cb5f7a2c4615a99a26c0566c66,NaN,NaN,Job Description Job #: 720298Apex Systems has...,Computer IT Services,Experienced (Non-Manager),NaN,NaN,NaN,construction pm,full time,charlottesville,va
a6a2b5e825b8ce1c3b517adb2497c5ed,NaN,NaN,"Part-Time, 4:30 pm - 9:30 pm, Mon - Fri Brookd...",Hotels Lodging Personal Household Services,Customer Support/Client Care,NaN,NaN,NaN,housekeeper,part time,austin,tx 78746
2f8bdf60db4d85627ab8f040e67aa78d,NaN,NaN,Aflac Insurance Sales Agent While a career in ...,Insurance,Customer Support/Client Care,NaN,NaN,NaN,aflac insurance sales agent insurance sales ag...,full time,berryville,va 22611


In [116]:
#----------------SD------------------
df_7_ = df_7['organization'].str.split(' ',n=1 ,expand=True)
df_7_= df_7_.rename(columns={0:'organization_', 1:'org_desc'})

# Here we create two columns organization and org_desc (organization description)

In [117]:
df_8 =df_7.join(df_7_, how='left').drop(columns =['organization','job_description'])
df_8.head(5)

,final_sal,initial_sal_,sector,benefits,period,period_cond,job,job_type,city,state,organization_,org_desc
uniq_id,,,,,,,,,,,,
58435fcab804439efdcaa7ecca0fd783,NaN,NaN,Experienced (Non-Manager),NaN,NaN,NaN,engineer,full time,dixon,ca,Altec,Industries
64d0272dc8496abfd9523a8df63c184c,NaN,NaN,Project/Program Management,NaN,NaN,NaN,shift supervisor,full time,camphill,pa,Retail,None
1e2637cb5f7a2c4615a99a26c0566c66,NaN,NaN,Experienced (Non-Manager),NaN,NaN,NaN,construction pm,full time,charlottesville,va,Computer,IT Services
a6a2b5e825b8ce1c3b517adb2497c5ed,NaN,NaN,Customer Support/Client Care,NaN,NaN,NaN,housekeeper,part time,austin,tx 78746,Hotels,Lodging Personal Household Services
2f8bdf60db4d85627ab8f040e67aa78d,NaN,NaN,Customer Support/Client Care,NaN,NaN,NaN,aflac insurance sales agent insurance sales ag...,full time,berryville,va 22611,Insurance,None


# sector

Same logic this column has information that can be use to predict someting else but has to be reduce to a tailored string that we can use

In [118]:

#-----------------SD------------------------
df_8_sector =  df_8['sector']
df_8_ = df_8_sector.replace('/', ' ').replace('(', ' ').str.split(' ', n=1, expand= True)
df_8_ =df_8_.rename(columns=({0:'sector_', 1:'sector_level'}))
df_8_.head(5)

,sector_,sector_level
uniq_id,,
58435fcab804439efdcaa7ecca0fd783,Experienced,(Non-Manager)
64d0272dc8496abfd9523a8df63c184c,Project/Program,Management
1e2637cb5f7a2c4615a99a26c0566c66,Experienced,(Non-Manager)
a6a2b5e825b8ce1c3b517adb2497c5ed,Customer,Support/Client Care
2f8bdf60db4d85627ab8f040e67aa78d,Customer,Support/Client Care


In [119]:
df_9 = df_8.join(df_8_, how='left').drop(columns='sector')
df_9.head()



,final_sal,initial_sal_,benefits,period,period_cond,job,job_type,city,state,organization_,org_desc,sector_,sector_level
uniq_id,,,,,,,,,,,,,
58435fcab804439efdcaa7ecca0fd783,NaN,NaN,NaN,NaN,NaN,engineer,full time,dixon,ca,Altec,Industries,Experienced,(Non-Manager)
64d0272dc8496abfd9523a8df63c184c,NaN,NaN,NaN,NaN,NaN,shift supervisor,full time,camphill,pa,Retail,None,Project/Program,Management
1e2637cb5f7a2c4615a99a26c0566c66,NaN,NaN,NaN,NaN,NaN,construction pm,full time,charlottesville,va,Computer,IT Services,Experienced,(Non-Manager)
a6a2b5e825b8ce1c3b517adb2497c5ed,NaN,NaN,NaN,NaN,NaN,housekeeper,part time,austin,tx 78746,Hotels,Lodging Personal Household Services,Customer,Support/Client Care
2f8bdf60db4d85627ab8f040e67aa78d,NaN,NaN,NaN,NaN,NaN,aflac insurance sales agent insurance sales ag...,full time,berryville,va 22611,Insurance,None,Customer,Support/Client Care


In [120]:
df_9.dtypes
# At this point the schemas of the dataset are objects but we are going to need to clean the content of the first to columns final_sal and initial_sal to change it to float
# why, because salaries are float or int values 

final_sal        object
initial_sal_     object
benefits         object
period           object
period_cond      object
job              object
job_type         object
city             object
state            object
organization_    object
org_desc         object
sector_          object
sector_level     object
dtype: object

In [121]:
#------------------CD-----------------

df_9['final_sal'] = df_9['final_sal'].str.strip()
df_9['initial_sal_'] = df_9['initial_sal_'].str.strip()
#Lets delete some spaces at the begining and end of the string, rememenber this two columns still are objects

In [122]:
def replace_char(data):
    return data.replace(',', '', regex = True).str.replace('\D', '', regex=True)

df_9['final_sal'] = replace_char(df_9['final_sal'])
df_9['initial_sal_'] = replace_char(df_9['initial_sal_'])
df_9.head(5)

# In these both columns final_sal and initial_sal_ we could find characteres because they came from a string so, in order to avoid convertions problems is better to make sure
# that all characteres be replace with ''

,final_sal,initial_sal_,benefits,period,period_cond,job,job_type,city,state,organization_,org_desc,sector_,sector_level
uniq_id,,,,,,,,,,,,,
58435fcab804439efdcaa7ecca0fd783,NaN,NaN,NaN,NaN,NaN,engineer,full time,dixon,ca,Altec,Industries,Experienced,(Non-Manager)
64d0272dc8496abfd9523a8df63c184c,NaN,NaN,NaN,NaN,NaN,shift supervisor,full time,camphill,pa,Retail,None,Project/Program,Management
1e2637cb5f7a2c4615a99a26c0566c66,NaN,NaN,NaN,NaN,NaN,construction pm,full time,charlottesville,va,Computer,IT Services,Experienced,(Non-Manager)
a6a2b5e825b8ce1c3b517adb2497c5ed,NaN,NaN,NaN,NaN,NaN,housekeeper,part time,austin,tx 78746,Hotels,Lodging Personal Household Services,Customer,Support/Client Care
2f8bdf60db4d85627ab8f040e67aa78d,NaN,NaN,NaN,NaN,NaN,aflac insurance sales agent insurance sales ag...,full time,berryville,va 22611,Insurance,None,Customer,Support/Client Care


In [123]:
def change_to_float(data):
    return data.replace('',np.nan).astype('float64').fillna(0)

df_9['final_sal'] = change_to_float(df_9['final_sal'])
df_9['initial_sal_'] = change_to_float(df_9['initial_sal_'])

df_9.head(5)

# In this chunk of code we change the initial salary and final salary to float because we need to create a mean of thiso two salaries that migth be used as a label


,final_sal,initial_sal_,benefits,period,period_cond,job,job_type,city,state,organization_,org_desc,sector_,sector_level
uniq_id,,,,,,,,,,,,,
58435fcab804439efdcaa7ecca0fd783,0.0,0.0,NaN,NaN,NaN,engineer,full time,dixon,ca,Altec,Industries,Experienced,(Non-Manager)
64d0272dc8496abfd9523a8df63c184c,0.0,0.0,NaN,NaN,NaN,shift supervisor,full time,camphill,pa,Retail,None,Project/Program,Management
1e2637cb5f7a2c4615a99a26c0566c66,0.0,0.0,NaN,NaN,NaN,construction pm,full time,charlottesville,va,Computer,IT Services,Experienced,(Non-Manager)
a6a2b5e825b8ce1c3b517adb2497c5ed,0.0,0.0,NaN,NaN,NaN,housekeeper,part time,austin,tx 78746,Hotels,Lodging Personal Household Services,Customer,Support/Client Care
2f8bdf60db4d85627ab8f040e67aa78d,0.0,0.0,NaN,NaN,NaN,aflac insurance sales agent insurance sales ag...,full time,berryville,va 22611,Insurance,None,Customer,Support/Client Care


In [124]:
df_9.dtypes

final_sal        float64
initial_sal_     float64
benefits          object
period            object
period_cond       object
job               object
job_type          object
city              object
state             object
organization_     object
org_desc          object
sector_           object
sector_level      object
dtype: object

In [125]:
def calculo_mean(initial, final):
    
    if initial == 0.0:
        return final
    elif final == 0.0:
        return initial
    else:
        return (initial+final)/2
               
        
df_9['mean_salary'] = df_9.apply(lambda x: calculo_mean(x['initial_sal_'] , x['final_sal']) , axis=1)
df_9.head(5)
# We need to create a mena value that might be useful for feature analisys bur what makes this process interesting is that some values in both initial_salary and final_salary
# are 0.0 so if we try to calculate mean fron (inital_salary+final_alary)/2 the result and one of this is 0 the result is going to be wrong 
# that's why we have to create this function
        


,final_sal,initial_sal_,benefits,period,period_cond,job,job_type,city,state,organization_,org_desc,sector_,sector_level,mean_salary
uniq_id,,,,,,,,,,,,,,
58435fcab804439efdcaa7ecca0fd783,0.0,0.0,NaN,NaN,NaN,engineer,full time,dixon,ca,Altec,Industries,Experienced,(Non-Manager),0.0
64d0272dc8496abfd9523a8df63c184c,0.0,0.0,NaN,NaN,NaN,shift supervisor,full time,camphill,pa,Retail,None,Project/Program,Management,0.0
1e2637cb5f7a2c4615a99a26c0566c66,0.0,0.0,NaN,NaN,NaN,construction pm,full time,charlottesville,va,Computer,IT Services,Experienced,(Non-Manager),0.0
a6a2b5e825b8ce1c3b517adb2497c5ed,0.0,0.0,NaN,NaN,NaN,housekeeper,part time,austin,tx 78746,Hotels,Lodging Personal Household Services,Customer,Support/Client Care,0.0
2f8bdf60db4d85627ab8f040e67aa78d,0.0,0.0,NaN,NaN,NaN,aflac insurance sales agent insurance sales ag...,full time,berryville,va 22611,Insurance,None,Customer,Support/Client Care,0.0


In [126]:
df_9[df_9['mean_salary']!=0]

,final_sal,initial_sal_,benefits,period,period_cond,job,job_type,city,state,organization_,org_desc,sector_,sector_level,mean_salary
uniq_id,,,,,,,,,,,,,,
b43c077756d5a326c4854e1399fd2464,1300.0,900.0,,hour,,primrose private preschool teacher,full time,houston,tx 77098,Education,None,Entry,Level,1100.0
d8491fcefe14d1398de419984dccf427,9500000.0,8000000.0,,year,,superintendent,full time,houston,tx,Construction,Industrial Facilities InfrastructureConstru...,Building,Construction/Skilled Trades,8750000.0
779bb4c9bf038b7fb775134736d36fd4,10000000.0,7000000.0,,year,,sr process engineer,full time,sr. process engineer,manufacturing,Chicago,IL,Engineering,None,8500000.0
ceb44cca7cd280adcb0c84c20f3c6c21,7200000.0,6800000.0,,year,,rf system tech,full time temporary,rf system technician,field service,Oklahoma,City OK,Engineering,None,7000000.0
eea9b50afc4fece9f9d6ff0dbf659784,6500000.0,5800000.0,,year,,japanese,full time,charleston,wv,Manufacturing,Other,Experienced,(Non-Manager),6150000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4e2b0a3e9fe5f8721f6ab4692823d9a9,0.0,25000.0,commisson depending on sales volume,week,7-9,route sales rep,full time,cincinnati,oh 45202,All,None,Student,(High School),25000.0
ac3c743eb7612d90bfe80a3a30d3d8be,6000000.0,5500000.0,"benefits package, life insurance, 401k, reloca...",year,ful,assistant quality mgr,full time,cincinnati,oh 45202,Automotive,Parts Mfg,Experienced,(Non-Manager),5750000.0
8e9f8638556bc1fd671bb99f4f01ac4d,8500000.0,7500000.0,competitive base salary plus lucrative bonus p...,year,highl,executive chef west chester ohio,full time,west chester,oh,All,None,Manager,(Manager/Supervisor of Staff),8000000.0


### Final dataset

Now that the dataset is clean in structure and data we have to make an inputation of the salary column for those values thar are Nan, at this moment I don't know if it's gonna work beacuase the amount of Nan Values is more than 83%, but lets try just for fun!!!! :)

#### First I want to organiza the dataset and have the information about nwe column created

- `initial_sal_`  Initial salary offered
- `final_sal`	    Final salary offered
- `mean_salary`   Mean salary obtained from the first two columns
- `period`	    Period of time year, month, week , hour that the payment of the salary is stablished
- `period_cond`   The contions offered acroding to the job
- `job`	        The position that the persons is taking
- `job_type`	    The type of job full time, part time....
- `city`	        City where the person has to work
- `state`	        The State of the city
- `organization_`	Name of the organization
- `org_desc`      Description of the organization
- `sector_`	    Sector that the organization belongs to 
- `sector_level`	Level of the position like manager or support
- `benefits`	    Benefits offered by the organization

### -----------------------------------  Finally we get our dataset Clean ---------------------------------------

In [127]:
clean_df = df_9[['initial_sal_','final_sal','mean_salary','period','period_cond','job','job_type','city', 'state','organization_','org_desc','sector_','sector_level','benefits']]
clean_df

,initial_sal_,final_sal,mean_salary,period,period_cond,job,job_type,city,state,organization_,org_desc,sector_,sector_level,benefits
uniq_id,,,,,,,,,,,,,,
58435fcab804439efdcaa7ecca0fd783,0.0,0.0,0.0,NaN,NaN,engineer,full time,dixon,ca,Altec,Industries,Experienced,(Non-Manager),NaN
64d0272dc8496abfd9523a8df63c184c,0.0,0.0,0.0,NaN,NaN,shift supervisor,full time,camphill,pa,Retail,None,Project/Program,Management,NaN
1e2637cb5f7a2c4615a99a26c0566c66,0.0,0.0,0.0,NaN,NaN,construction pm,full time,charlottesville,va,Computer,IT Services,Experienced,(Non-Manager),NaN
a6a2b5e825b8ce1c3b517adb2497c5ed,0.0,0.0,0.0,NaN,NaN,housekeeper,part time,austin,tx 78746,Hotels,Lodging Personal Household Services,Customer,Support/Client Care,NaN
2f8bdf60db4d85627ab8f040e67aa78d,0.0,0.0,0.0,NaN,NaN,aflac insurance sales agent insurance sales ag...,full time,berryville,va 22611,Insurance,None,Customer,Support/Client Care,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7502ee8f0d324f86334c531fa8bcf663,0.0,0.0,0.0,NaN,NaN,accountant,full time,cincinnati,oh 45249,Healthcare,Services,Entry,Level,NaN
9796e104240789dd33cc436f6c383892,0.0,0.0,0.0,NaN,NaN,licensed practical nurse lpn,full time,cincinnati,oh 45236,Healthcare,Services Other Not Classified,Medical/Health,None,NaN
abd9ad3e0ec3c934b5a59f3776012865,0.0,0.0,0.0,NaN,NaN,immediate customer service position,part time,cincinnati,oh 45202,All,None,Entry,Level,NaN


In [128]:
shape=df_.shape
clean_df.isnull().sum()/shape[0]*100

initial_sal_      0.000000
final_sal         0.000000
mean_salary       0.000000
period           86.055541
period_cond      86.055541
job               0.000000
job_type          0.000000
city              0.000000
state             6.727442
organization_     0.000000
org_desc         19.540812
sector_           0.000000
sector_level     28.631721
benefits         86.055541
dtype: float64

In [129]:
# Skimpy is a librarie that create a framework for us to help with general observation 
#pip install skimpy

from skimpy import skim
skim(clean_df)

# Mosto of the NA values comes from subsets of original columns, this important to have under considerations once we make an imputation 

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 11847  │ │ string      │ 11    │                                                          │
│ │ Number of columns │ 14     │ │ float64     │ 3     │                                                          │
│ └───────────────────┴────────┘ └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━┓  │
│ ┃ column_name       ┃ NA   ┃ NA %    ┃ mean      ┃ sd         ┃ p0   ┃ p25   ┃ p75   ┃ p100        ┃ hist    ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━┩  │
│ │ initial_sal_      │    0 │       0 │    460000 │    1800000 │    0 │     0 │     0 │    22000000 │    █    │  │
│ │ final_sal         │    0 │       0 │    590000 │    2600000 │    0 │     0 │     0 │    90000000 │    █    │  │
│ │ mean_salary       │    0 │       0 │    570000 │    2200000 │    0 │     0 │     0 │    50000000 │    █    │  │
│ └───────────────────┴──────┴─────────┴───────────┴────────────┴──────┴───────┴───────┴─────────────┴─────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name                ┃ NA          ┃ NA %       ┃ words per row              ┃ total words           ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ period                     │       10000 │         86 │                       0.14 │                  1700 │  │
│ │ period_cond                │       10000 │         86 │                       0.14 │                  1700 │  │
│ │ job                        │           0 │          0 │                       0.14 │                  1700 │  │
│ │ job_type                   │           0 │          0 │                       0.14 │                  1700 │  │
│ │ city                       │           0 │          0 │                       0.14 │                  1700 │  │
│ │ state                      │         800 │        6.7 │                       0.14 │                  1700 │  │
│ │ organization_              │           0 │          0 │                       0.14 │                  1700 │  │
│ │ org_desc                   │        2300 │         20 │                       0.14 │                  1700 │  │
│ │ sector_                    │           0 │          0 │                       0.14 │                  1700 │  │
│ │ sector_level               │        3400 │         29 │                       0.14 │                  1700 │  │
│ │ benefits                   │       10000 │         86 │                       0.14 │                  1700 │  │
│ └────────────────────────────┴─────────────┴────────────┴────────────────────────────┴───────────────────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯

# Imputation — the process of replacing missing data with substituted values.

I'm going to dedicate this imputation process to another project, i'll be soon placing its name here

In [130]:
clean_df.to_csv('clean_df.csv')